# Stance Detection for the Fake News Challenge

## Identifying Textual Relationships with Deep Neural Nets

### Check the problem context [here](https://drive.google.com/open?id=1KfWaZyQdGBw8AUTacJ2yY86Yxgw2Xwq0).

### Download files required for the project from [here](https://drive.google.com/open?id=10yf39ifEwVihw4xeJJR60oeFBY30Y5J8).

## Step1: Load the given dataset  

1. Mount the google drive

2. Import Glove embeddings

3. Import the test and train datasets

### Mount the google drive to access required project files

Run the below commands

In [0]:
from google.colab import drive

In [3]:
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Path for Project files on google drive

**Note:** You need to change this path according where you have kept the files in google drive. 

In [0]:
project_path = "/content/drive/My Drive/nlp_project_fake_news/"

### Loading the Glove Embeddings

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'glove.6B.zip', 'r') as z:
  z.extractall()

# Load the dataset [5 Marks]

1. Using [read_csv()](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) in pandas load the given train datasets files **`train_bodies.csv`** and **`train_stances.csv`**

2. Using [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) command in pandas merge the two datasets based on the Body ID. 

Note: Save the final merged dataset in a dataframe with name **`dataset`**.

In [0]:
import pandas as pd
bodies = pd.read_csv('/content/drive/My Drive/nlp_project_fake_news/train_bodies.csv')

In [0]:
stances = pd.read_csv('/content/drive/My Drive/nlp_project_fake_news/train_stances.csv')

In [8]:
bodies.head()

,Body ID,articleBody
0,0,A small meteorite crashed into a wooded area i...
1,4,Last week we hinted at what was to come as Ebo...
2,5,(NEWSER) – Wonder how long a Quarter Pounder w...
3,6,"Posting photos of a gun-toting child online, I..."
4,7,At least 25 suspected Boko Haram insurgents we...


In [9]:
stances.head()

,Headline,Body ID,Stance
0,Police find mass graves with at least '15 bodi...,712,unrelated
1,Hundreds of Palestinians flee floods in Gaza a...,158,agree
2,"Christian Bale passes on role of Steve Jobs, a...",137,unrelated
3,HBO and Apple in Talks for $15/Month Apple TV ...,1034,unrelated
4,Spider burrowed through tourist's stomach and ...,1923,disagree


In [0]:
dataset = bodies.merge(stances, on = 'Body ID')


<h2> Check1:</h2>
  
<h3> You should see the below output if you run `dataset.head()` command as given below </h3>

In [11]:
dataset.head()

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated


## Step2: Data Pre-processing and setting some hyper parameters needed for model


#### Run the code given below to set the required parameters.

1. `MAX_SENTS` = Maximum no.of sentences to consider in an article.

2. `MAX_SENT_LENGTH` = Maximum no.of words to consider in a sentence.

3. `MAX_NB_WORDS` = Maximum no.of words in the total vocabualry.

4. `MAX_SENTS_HEADING` = Maximum no.of sentences to consider in a heading of an article.

In [0]:
MAX_NB_WORDS = 20000
MAX_SENTS = 20
MAX_SENTS_HEADING = 1
MAX_SENT_LENGTH = 20
VALIDATION_SPLIT = 0.2

### Download the `Punkt` from nltk using the commands given below. This is for sentence tokenization.

For more info on how to use it, read [this](https://stackoverflow.com/questions/35275001/use-of-punktsentencetokenizer-in-nltk).



In [14]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Tokenizing the text and loading the pre-trained Glove word embeddings for each token  [5 marks] 

Keras provides [Tokenizer API](https://keras.io/preprocessing/text/) for preparing text. Read it before going any further.

#### Import the Tokenizer from keras preprocessing text

In [15]:
from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.


#### Initialize the Tokenizer class with maximum vocabulary count as `MAX_NB_WORDS` initialized at the start of step2. 

In [0]:
t = Tokenizer(num_words=MAX_NB_WORDS)

#### Now, using fit_on_texts() from Tokenizer class, lets encode the data 

Note: We need to fit articleBody and Headline also to cover all the words.

In [0]:
texts = dataset['articleBody'] + dataset['Headline']
t.fit_on_texts(texts)


In [20]:
t

#### fit_on_texts() gives the following attributes in the output as given [here](https://faroit.github.io/keras-docs/1.2.2/preprocessing/text/).

* **word_counts:** dictionary mapping words (str) to the number of times they appeared on during fit. Only set after fit_on_texts was called.

* **word_docs:** dictionary mapping words (str) to the number of documents/texts they appeared on during fit. Only set after fit_on_texts was called.

* **word_index:** dictionary mapping words (str) to their rank/index (int). Only set after fit_on_texts was called.

* **document_count:** int. Number of documents (texts/sequences) the tokenizer was trained on. Only set after fit_on_texts or fit_on_sequences was called.



### Now, tokenize the sentences using nltk sent_tokenize() and encode the senteces with the ids we got form the above `t.word_index`

Initialise 2 lists with names `texts` and `articles`.

```
texts = [] to store text of article as it is.

articles = [] split the above text into a list of sentences.
```

In [0]:
texts = []
articles = []

In [0]:
texts = dataset['articleBody']

In [0]:
from nltk.tokenize import sent_tokenize
for items in texts.iteritems(): 
    articles.append(sent_tokenize(items[1]))

## Check 2:

first element of texts and articles should be as given below. 

In [0]:
texts[0]

'A small meteorite crashed into a wooded area in Nicaragua\'s capital of Managua overnight, the government said Sunday. Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city\'s airport, the Associated Press reports. \n\nGovernment spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth." House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports. \nMurillo said Nicaragua will ask international experts to help local scientists in understanding what happened.\n\nThe crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee. He said it is still not clear if the meteorite disintegrated or was buried.\n\nHumbe

In [23]:
articles[0]

["A small meteorite crashed into a wooded area in Nicaragua's capital of Managua overnight, the government said Sunday.",
 "Residents reported hearing a mysterious boom that left a 16-foot deep crater near the city's airport, the Associated Press reports.",
 'Government spokeswoman Rosario Murillo said a committee formed by the government to study the event determined it was a "relatively small" meteorite that "appears to have come off an asteroid that was passing close to Earth."',
 'House-sized asteroid 2014 RC, which measured 60 feet in diameter, skimmed the Earth this weekend, ABC News reports.',
 'Murillo said Nicaragua will ask international experts to help local scientists in understanding what happened.',
 'The crater left by the meteorite had a radius of 39 feet and a depth of 16 feet,  said Humberto Saballos, a volcanologist with the Nicaraguan Institute of Territorial Studies who was on the committee.',
 'He said it is still not clear if the meteorite disintegrated or was bu

# Now iterate through each article and each sentence to encode the words into ids using t.word_index  [5 marks] 

Here, to get words from sentence you can use `text_to_word_sequence` from keras preprocessing text.

1. Import text_to_word_sequence

2. Initialize a variable of shape (no.of articles, MAX_SENTS, MAX_SENT_LENGTH) with name `data` with zeros first (you can use numpy [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html) to initialize with all zeros)and then update it while iterating through the words and sentences in each article.

In [0]:
from keras.preprocessing.text import text_to_word_sequence
import numpy as np

In [0]:
data = np.zeros((len(articles), MAX_SENTS, MAX_SENT_LENGTH)).astype(int)

In [0]:
for pos1, item in enumerate(articles):
    for pos2, sent in enumerate(item):
        if(sent is not None) & (pos2 < MAX_SENTS):
            temp = text_to_word_sequence(sent)
            for pos3, word in enumerate(temp):
                if (pos3 < MAX_SENT_LENGTH):
                    data[pos1][pos2][pos3] = t.word_index.get(word, 0)

### Check 3:

Accessing first element in data should give something like given below.

In [28]:
data[0, :, :]

array([[    3,   485,   433,  7204,    81,     3,  3732,   331,     5,
         3888,   350,     4,  1432,  2956,     1,    89,    12,   464,
            0,     0],
       [  757,    95,  1045,     3,  2675,  1750,     7,   188,     3,
         1217,  1074,  2026,   698,   158,     1,  3029,   449,     1,
          555,   243],
       [   89,  1065,  4111,  2345,    12,     3,  1092,  3300,    19,
            1,    89,     2,  1791,     1,   529,  2005,    15,     9,
            3,  3107],
       [  186,  3639,   971,   202,  2553,    43,  6770,  1719,  1250,
            5, 13306, 17921,     1,   776,    31,   738,  3986,    67,
           85,     0],
       [ 2345,    12,  1584,    38,  1094,   351,   777,     2,   367,
          260,  1775,     5,  4447,    70,   494,     0,     0,     0,
            0,     0],
       [    1,   698,   188,    19,     1,   433,    32,     3,  7411,
            4,  2256,  1250,     6,     3,  5266,     4,  1217,  1250,
           12,  3359],
       [  

# Repeat the same process for the `Headings` as well. Use variables with names `texts_heading` and `articles_heading` accordingly. [5 marks] 

In [0]:
texts_headings = []
articles_headings = []
texts_headings = dataset['Headline']
for items in texts_headings.iteritems(): 
    articles_headings.append(sent_tokenize(items[1]))

In [0]:
data_heading = np.zeros((len(articles_headings), MAX_SENTS, MAX_SENT_LENGTH)).astype(int)

In [0]:
for pos1, item in enumerate(articles_headings):
    for pos2, sent in enumerate(item):
        if(sent is not None) & (pos2 < MAX_SENTS):
            temp = text_to_word_sequence(sent)
            for pos3, word in enumerate(temp):
                if (pos3 < MAX_SENT_LENGTH):
                    data_heading[pos1][pos2][pos3] = t.word_index.get(word, 0)

### Now the features are ready, lets make the labels ready for the model to process.

### Convert labels into one-hot vectors

You can use [get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) in pandas to create one-hot vectors.

In [0]:
labels = pd.get_dummies(dataset['Stance'])

In [34]:
labels.head()

,agree,disagree,discuss,unrelated
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1


### Check 4:

The shape of data and labels shoould match the given below numbers.

In [35]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (49972, 20, 20)
Shape of label tensor: (49972, 4)


### Shuffle the data

In [0]:
## get numbers upto no.of articles
indices = np.arange(data.shape[0])
## shuffle the numbers
np.random.shuffle(indices)

In [0]:
## shuffle the data
data = data[indices]
data_heading = data_heading[indices]
## shuffle the labels according to data
labels = labels.iloc[indices]

### Split into train and validation sets. Split the train set 80:20 ratio to get the train and validation sets.


Use the variable names as given below:

x_train, x_val - for body of articles.

x-heading_train, x_heading_val - for heading of articles.

y_train - for training labels.

y_val - for validation labels.



In [0]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(data, labels, test_size=0.2, random_state=10)

### Check 5:

The shape of x_train, x_val, y_train and y_val should match the below numbers.

In [105]:
print(x_train.shape)
print(y_train.shape)

print(x_val.shape)
print(y_val.shape)

(39977, 20, 20)
(39977, 4)
(9995, 20, 20)
(9995, 4)


### Create embedding matrix with the glove embeddings


Run the below code to create embedding_matrix which has all the words and their glove embedding if present in glove word list.

In [59]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('./glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

# create a weight matrix for words in training docs
embedding_matrix = np.zeros((len(word_index) + 1, 100))


for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

Loaded 400000 word vectors.


# Try the sequential model approach and report the accuracy score. [10 marks]  

### Import layers from Keras to build the model

In [0]:
import tensorflow as tf
from tensorflow import keras
from keras import backend as K
import numpy as np
from keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional, GlobalAveragePooling1D, Flatten


tf.set_random_seed(100)

In [0]:
x_train = np.reshape(x_train, (39977, 400))
x_val = np.reshape(x_val, (9995, 400))

In [110]:
x_train.shape

(39977, 400)

### Model

In [0]:
model = keras.Sequential()
model.add(keras.layers.Embedding(len(word_index) + 1, 100,  weights=[embedding_matrix], trainable=False))
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(32, activation="relu"))
model.add(keras.layers.Dense(16, activation="relu"))
model.add(keras.layers.Dense(4, activation="sigmoid"))

In [114]:
model.summary()

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, None, 100)         3364300   
_________________________________________________________________
global_average_pooling1d_16  (None, 100)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 64)                6464      
_________________________________________________________________
dropout_12 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_49 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_50 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_51 (Dense)             (None, 4)               

### Compile and fit the model

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [116]:
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs = 10, batch_size = 50, verbose = 1)

Train on 39977 samples, validate on 9995 samples
Epoch 1/10
39977/39977 [==============================] - 3s 68us/sample - loss: 0.8375 - acc: 0.7248 - val_loss: 0.8056 - val_acc: 0.7256
Epoch 2/10
39977/39977 [==============================] - 3s 63us/sample - loss: 0.8002 - acc: 0.7327 - val_loss: 0.8036 - val_acc: 0.7256
Epoch 3/10
39977/39977 [==============================] - 2s 62us/sample - loss: 0.7979 - acc: 0.7327 - val_loss: 0.8047 - val_acc: 0.7256
Epoch 4/10
39977/39977 [==============================] - 3s 65us/sample - loss: 0.7970 - acc: 0.7327 - val_loss: 0.8043 - val_acc: 0.7256
Epoch 5/10
39977/39977 [==============================] - 2s 62us/sample - loss: 0.7967 - acc: 0.7327 - val_loss: 0.8047 - val_acc: 0.7256
Epoch 6/10
39977/39977 [==============================] - 3s 65us/sample - loss: 0.7965 - acc: 0.7327 - val_loss: 0.8037 - val_acc: 0.7256
Epoch 7/10
39977/39977 [==============================] - 2s 62us/sample - loss: 0.7960 - acc: 0.7327 - val_loss: 0.8

In [0]:
i, val = np.where(y_val)
pred = model.predict_classes(x_val)

In [118]:
from sklearn.metrics import accuracy_score

print ('Model Accuracy: ', accuracy_score(val, pred))

Model Accuracy:  0.7255627813906953


## Build the same model with attention layers included for better performance (Optional)

## Fit the model and report the accuracy score for the model with attention layer (Optional)